# Import The required liberaries

In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Upload And Clean Dataset

In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

# Split the Dataset into features and labels

In [7]:
concrete_data_columns = concrete_data.columns

#features
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength

#label
target = concrete_data['Strength']

# Basic Regression Model

In [8]:
# define regression model
def regression_model(input_cols):
    
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(input_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [9]:
def model(predictors,target,epochs,random_state):
    
    # Split the data into train and test
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3, random_state=random_state)
    
    # Train the model
    model = regression_model(predictors_train.shape[1])
    
    # fit the model on the training data
    hist = model.fit(predictors_train, target_train, epochs=epochs, verbose=0)
    
    # find prediction on the test data
    mse_loss = model.evaluate(predictors_test, target_test, verbose=0)
    
    # find target prediction using the model
    target_pred = model.predict(predictors_test)
    
    # calculate mean square error between model prediction and result
    mean_square_error = mean_squared_error(target_test, target_pred)
    
    return mean_square_error,mse_loss,hist

# Train and Test Model

In [10]:
def train_test_model(total_mean_squared_errors, epochs, predictors,target):
    mean_squared_errors_list = []
    history_list = []
    MSE_list = []
    for i in range(0, total_mean_squared_errors):
        mean_square_error,MSE,hist = model(predictors,target,epochs,random_state=i)
        mean_squared_errors_list.append(mean_square_error)
        history_list.append(hist)
        MSE_list.append(MSE)
    return mean_squared_errors_list,history_list,MSE_list

In [11]:
total_mean_squared_errors = 1
epochs = 50
mean_squared_errors_list,history_list,MSE_list =  train_test_model(total_mean_squared_errors, epochs, predictors,target)
for i in range(len(MSE_list)):
    print("MSE "+str(i+1)+": "+str(MSE_list[i]))
mean_squared_errors_list = np.array(mean_squared_errors_list)
mean = np.mean(mean_squared_errors_list)
standard_deviation = np.std(mean_squared_errors_list)   
print(history_list)
print('\n')
print("Mean And Standard Deviation of 1 epochs "+ "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 190.01487119606784


Mean And Standard Deviation of 1 epochs 

Mean: 190.0148691743412
Standard Deviation: 0.0


In [12]:
total_mean_squared_errors = 50
mean_squared_errors_list,history_list,MSE_list = train_test_model(total_mean_squared_errors, epochs, predictors,target)
for i in range(len(MSE_list)):
    print("MSE "+str(i+1)+": "+str(MSE_list[i]))
mean_squared_errors_list = np.array(mean_squared_errors_list)
mean = np.mean(mean_squared_errors_list)
standard_deviation = np.std(mean_squared_errors_list)    
print('\n')
print("Mean And Standard Deviation of 50 epochs "+ "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 76.61142530719053
MSE 2: 346.0240184203706
MSE 3: 194.29120396487536
MSE 4: 148.88497833449478
MSE 5: 349.5315372010265
MSE 6: 1316.4743498274424
MSE 7: 572.4977324155542
MSE 8: 97.08491582777894
MSE 9: 193.39590133123798
MSE 10: 2029.2856753451154
MSE 11: 120.68533098041819
MSE 12: 1459.7666987446905
MSE 13: 242.86026415778596
MSE 14: 226.23383343644127
MSE 15: 102.46946432366727
MSE 16: 122.38937829767616
MSE 17: 184.92769807982214
MSE 18: 327.76820309339604
MSE 19: 759.2158906313208
MSE 20: 198.06234706644102
MSE 21: 165.15731934976424
MSE 22: 103.5550686734394
MSE 23: 229.8396998038184
MSE 24: 111.04825977214331
MSE 25: 145.50094434127067
MSE 26: 103.97556682697778
MSE 27: 361.91911854172986
MSE 28: 553.9531378391106
MSE 29: 129.03130585013085
MSE 30: 223.18483573641979
MSE 31: 280.61572374263625
MSE 32: 303.6366746032122
MSE 33: 1113.296397385088
MSE 34: 531.0007592852447
MSE 35: 138.73944985596492
MSE 36: 309.4087565400454
MSE 37: 125.33481452225867
MSE 38: 232.52949593059